In [ ]:
import Utils
import pyDOE
import pandas as pd
import numpy as np
import scipy.stats.distributions as dist
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from collections import namedtuple
from sklearn.preprocessing import MinMaxScaler

## Helper Functions

In [2]:
''' Support Vector Regression for Robust Regularization'''
def SVR_minimax(train_data,test_data,Hyper):
    gam, reg= Hyper
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    gpr = SVR(gamma=gam ,C=reg, epsilon=0.25,max_iter=1500).fit(
        scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,11])
    pred = gpr.predict(scaler.transform(test_data))
    return gpr,pred

''' Support Vector Regression for Robust Composition'''
def SVR_composite(train_data,test_data,Hyper):
    gam, reg = Hyper
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    gpr = SVR(gamma=gam,  C=reg, epsilon=0.2
             ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    return gpr,pred


''' Latin HyperCube Sampling Design of Experiment for Hyper_Parameters Optimization'''
def DOE_Hyper():
    lhd = pyDOE.lhs(n=2, samples=1500, criterion='m')
    X1, X2 = lhd[:,0], lhd[:,1] 
    X1 = Utils.linearscaletransform(X1,range_out=(0.00001,1e2))
    X2 = Utils.linearscaletransform(X2,range_out=(0.0001 , 1000))
    Hyper_Parameters = pd.DataFrame()
    Hyper_Parameters['Gamma'] = pd.Series(X1)
    Hyper_Parameters['Regularization'] = pd.Series(X2)
    return Hyper_Parameters

## Load Training and Test Data Set initially Generated

In [3]:
train = pd.read_csv(sys.path[1][:-4]+str('\\Training_Data_Sets\\50Samples.csv')).iloc[:,1:]
test = pd.read_csv(sys.path[1][:-4]+str('\\Test_Data_Sets\\750Samples.csv')).iloc[:,1:]
true_minimax = np.array(test.True_Minimax)
true_composite = np.array(test.True_Composite)

## SVR

In [ ]:
X = [50,100,150,200,250,300,350,400,450,500]
Kriging_minimax = np.zeros([len(X),2])
Kriging_composite = np.zeros([len(X),2])
for k in range(len(X)):
    print (X[k])
    train = pd.read_csv(sys.path[1][:-4]+str('\\Training_Data_Sets\\'+str(X[k])+'Samples.csv')).iloc[:,1:]
    test = pd.read_csv(sys.path[1][:-4]+str('\\Test_Data_Sets\\750Samples.csv')).iloc[:,1:]
    true_minimax = np.array(test.True_Minimax)
    true_composite = np.array(test.True_Composite)
    Hyper_Parameters = DOE_Hyper()
    mean_abs_error_minimax = np.zeros(Hyper_Parameters.shape[0])
    mean_abs_error_composite = np.zeros(Hyper_Parameters.shape[0])
    for i in range(Hyper_Parameters.shape[0]):
        temp = np.array(Hyper_Parameters.iloc[i,:])
        model_minimax,pred_k_m = SVR_minimax(train,test.iloc[:,:10],temp)
        model_composite,pred_k_c = SVR_composite(train,test.iloc[:,:10],temp)
        mean_abs_error_minimax[i] = np.mean((abs(true_minimax-pred_k_m) / abs(true_minimax) ) * 100)
        mean_abs_error_composite[i] = np.mean((abs( true_composite-pred_k_c) / abs(true_composite) ) * 100)
    Hyper_Parameters ['Mean_Error_Minimax'] = pd.Series(mean_abs_error_minimax)
    Hyper_Parameters ['Mean_Error_Composite'] = pd.Series(mean_abs_error_composite)
    Kriging_minimax [k,:] = np.array(Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Minimax.values.argmin(),:-2])
    Kriging_composite [k,:] = np.array(Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Composite.values.argmin(),:-2])

In [5]:
print ('Best Configuration of Hyper Parameters for Robust Regularization')
Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Minimax.values.argmin(),:-1]

Best Configuration of Hyper Parameters for Robust Regularization


Gamma                   0.085344
Regularization        703.315585
Mean_Error_Minimax      1.786918
Name: 558, dtype: float64

In [6]:
print ('Best Configuration of Hyper Parameters for Robust Regularization')
Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Composite.values.argmin(),[0,1,3]]

Best Configuration of Hyper Parameters for Robust Regularization


Gamma                     0.085344
Regularization          703.315585
Mean_Error_Composite      0.764393
Name: 558, dtype: float64

In [7]:
Hyper_Parameters.to_csv('Result_Hyper_Params.csv')

In [8]:
pd.DataFrame(Kriging_composite).to_csv('composite.csv')
pd.DataFrame(Kriging_minimax).to_csv('minimax.csv')

In [9]:
minimax = pd.read_csv('minimax.csv').iloc[:,1:]
composite = pd.read_csv('composite.csv').iloc[:,1:]

In [10]:
np.array(composite)

array([[3.12253184e-01, 6.09347260e+02],
       [3.28014021e-01, 4.19498608e+02],
       [2.39611321e-01, 4.59765582e+02],
       [2.06132759e-01, 5.44014106e+02],
       [1.07259239e-01, 6.81120508e+02],
       [2.07004671e-01, 4.98124611e+02],
       [1.86361488e-01, 5.90693615e+02],
       [1.38763994e-01, 6.08545310e+02],
       [9.64341034e-02, 5.40008587e+02],
       [8.53443233e-02, 7.03315585e+02]])

In [11]:
np.array(minimax)

array([[5.94985993e-01, 1.65813533e+02],
       [4.74862477e-01, 1.86038992e+02],
       [1.37755283e-01, 3.66997650e+02],
       [1.32896676e-01, 2.42166170e+02],
       [1.70287434e-01, 1.79163720e+02],
       [2.07004671e-01, 4.98124611e+02],
       [9.89057033e-02, 9.41770798e+02],
       [8.26062363e-02, 7.28446785e+02],
       [9.64341034e-02, 5.40008587e+02],
       [8.53443233e-02, 7.03315585e+02]])